In [1]:
import torch
import matplotlib.pyplot as plt
import pandas as pd
import time
import random
torch.set_default_tensor_type(torch.DoubleTensor)

On charge notre jeu de données

In [6]:
data = pd.read_csv('https://joon-kwon.github.io/optim-mathsv/blogData_train.csv', header=None)
X = torch.from_numpy(data.values[:, :-1])
y = torch.from_numpy(data.values[:, -1].reshape(-1))
d = len(X[0])
n = len(X)

On définit notre fonction objectif qui correspond à une régression linéaire aux moindres carrés. On peut éventuellement passer en second argument un mini-batch (sous la forme d'une liste contenant les indices correspondants), si on souhaite ne calculer la valeur que sur ce mini-batch.

In [7]:
def f(x, minibatch=None):
    if minibatch is None:
        return torch.mean((torch.tensordot(X, x, dims=([1], [0])) - y)**2)
    else:
        return torch.mean((torch.tensordot(X[minibatch], x, dims=([1], [0])) - y[minibatch])**2)

La différentiation automatique sous PyTorch fonctionne de la façon suivante.

In [8]:
x = torch.zeros(d) # Le point en lequel on souhaite calculer un gradient
x.requires_grad = True # On indique qu'on va calculer un gradient par rapport à cette variable
value = f(x) # On calcule d'abord la fonction elle-même en ce point
value.backward() # On demande à pytorch de calculer le gradient
print(x.grad) # Le gradient correspondant est alors stocké dans x.grad
x = x.detach() # On arrête la fonctionnalité de différentiation automatique (attention à sauver x.grad auparavant)

tensor([-3.4303e+03, -2.6301e+03, -3.2311e+01, -1.6469e+04, -2.9147e+03,
        -1.4161e+03, -1.6398e+03, -3.8449e-01, -1.1305e+04, -9.8679e+02,
        -1.2496e+03, -1.6925e+03,  0.0000e+00, -1.1273e+04, -5.8779e+02,
        -2.8543e+03, -2.0955e+03, -3.2311e+01, -1.2334e+04, -2.5479e+03,
        -1.6657e+02, -2.4195e+03,  8.7663e+03, -1.1119e+04,  7.3602e+00,
        -3.0588e+01, -2.9762e+01, -2.2749e-02, -2.0534e+02, -2.5787e+01,
        -1.1696e+01, -2.0189e+01,  0.0000e+00, -1.5924e+02, -5.0265e+00,
        -1.1143e+01, -2.0311e+01,  0.0000e+00, -1.5923e+02, -8.3974e-03,
        -2.8544e+01, -2.8059e+01, -2.2749e-02, -1.8192e+02, -1.9055e+01,
        -5.5318e-01, -3.0101e+01,  1.3194e+02, -1.5385e+02, -7.3859e-03,
        -3.1678e+03, -2.2541e+03, -6.9056e+02, -2.7200e+03, -1.5636e+03,
        -2.9403e+01, -1.9855e+01, -6.8439e+00, -2.8239e+01, -1.3012e+01,
        -2.3238e+02, -5.2429e+04, -3.5727e-02, -3.3457e+00,  0.0000e+00,
        -3.7788e-03, -7.0982e+00, -2.7347e+00, -5.2

*Question 1*: Utiliser la différentiation automatique pour implémenter une descente de gradient (à pas constant). Faire cesser les itérations au bout de 5 secondes (par exemple). Afficher la valeur de la fonction objectif en la dernière itérée. Essayer par tâtonnements différentes valeurs pour le pas.

On peut par exemple se donner un mini-batch de taille 100 de la façon suivante.

In [10]:
minibatch = random.sample(range(0,n), 100)
print(minibatch)

[17391, 20778, 20494, 14524, 46604, 44562, 50590, 18505, 38736, 22943, 33880, 34309, 41886, 29594, 46134, 1353, 37183, 30538, 40826, 26178, 24898, 33808, 33562, 48087, 693, 51388, 41985, 37972, 19327, 33833, 7831, 23374, 49318, 20205, 25006, 32624, 46555, 39514, 20944, 41166, 20836, 51194, 17192, 3086, 35423, 19537, 25987, 36673, 17060, 5648, 28480, 4279, 8687, 28170, 12191, 24009, 26727, 26653, 47064, 43525, 24524, 46250, 50564, 48769, 22311, 37564, 4835, 35131, 4547, 18556, 31830, 49446, 20200, 43030, 40357, 13850, 17392, 5302, 10533, 36936, 26593, 22285, 50059, 39057, 41078, 27185, 29046, 1578, 583, 42207, 31632, 50181, 22980, 12059, 27595, 1055, 52235, 25219, 17524, 30195]


*Question 2*: Modifier le code de la question 1 pour utiliser des mini-batchs. Essayer différentes tailles de mini-batchs. Commenter les résultats.

PyTorch propose des implémentations d'algorithmes d'optimisation qui peuvent être utilisés de la façon suivante.
`lr` correspond au pas. La prodécure `.step()` opère l'itération de l'algorithme d'optimisation en utilisant le gradient qui vient d'être calculé, et met à jour la valeur de la variable `x`.

In [13]:
x = torch.zeros(d)
x.requires_grad = True
optimizer = torch.optim.SGD([x], lr=1e-9)
for t in range(100):
    optimizer.zero_grad()
    value = f(x)
    value.backward()
    optimizer.step()
f(x)

tensor(1377.3830, grad_fn=<MeanBackward0>)

*Question 3*: À la place de SGD (qui correspond tout simplement à la descente de gradient), utiliser les implémentations de RMSprop, Adam, etc. (voir https://pytorch.org/docs/stable/optim.html), éventuellement avec des mini-batchs, toujours en stoppant l'exécution au bout de 5 secondes, et en affichant la valeur de la fonction objectif en la dernière itéreé. Comparer les résultats obtenus.